In [1]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

(32, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [2]:

for index, row in df.iterrows():
    doc_id = row.doc_id

    author_id = row.author_id

    import ast
    authorList = ast.literal_eval(row.authorList)
    
    candidate = len(authorList)
    
    algo = "svc"
    
    test = str(algo) # change before run
    
    level = "word"
    
    iterations = 30

    dropout = 0.5

    samples = 3200

    dimensions = 200

    loc = authorList.index(author_id)

    printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
    printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
    printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))

    print("Current test: %s" % (str(printstate)))
    from sshtunnel import SSHTunnelForwarder
    with SSHTunnelForwarder(('144.214.121.15', 22),
                            ssh_username='ninadt',
                            ssh_password='Ninad123',
                            remote_bind_address=('localhost', 3306),
                            local_bind_address=('localhost', 3300)):
        import UpdateDB as db
        case = db.checkOldCNNDiff(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                  samples = samples, iterations = iterations, dropout = dropout,
                                  test = test, port = 3300)

    if case == False:
        
        print("Running: %12s" % (str(printstate)))
        
        import StyloNeural as Stylo
        (labels_index, history, train_acc_cnn, val_acc_cnn, samples) = Stylo.getResults(
            doc_id = doc_id, authorList = authorList[:], 
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

        (predY_cnn, testY_cnn) = Stylo.getTestResults(
            doc_id = doc_id, authorList = authorList[:], labels_index = labels_index,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )
            
        del Stylo

        from keras import backend as K
        K.clear_session()

        from IPython.display import clear_output

        clear_output()
        
        import StyloNeuralML as Stylo
        (labels_index, train_acc_ml, val_acc_ml, samples) = Stylo.getResults(
            doc_id = doc_id, authorList = authorList[:], algo = algo,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )
        
        (predY_ml, testY_ml) = Stylo.getTestResults(
            doc_id = doc_id, authorList = authorList[:], labels_index = labels_index, algo = algo,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )
            
        del Stylo

        from keras import backend as K
        K.clear_session()
            
        loc = testY_cnn
        
        test_acc_cnn = 0.0
        
        test_acc_cnn = predY_cnn[loc]

        test_bin_cnn = 0

        if(predY_cnn.tolist().index(max(predY_cnn)) == testY_cnn):
            test_bin_cnn = 1
            
        loc = testY_ml
        
        test_acc_ml = 0.0
        
        test_acc_ml = predY_ml[loc]

        test_bin_ml = 0

        if(predY_ml.tolist().index(max(predY_ml)) == testY_ml):
            test_bin_ml = 1
        
        from sshtunnel import SSHTunnelForwarder
        with SSHTunnelForwarder(('144.214.121.15', 22),
                                ssh_username='ninadt',
                                ssh_password='Ninad123',
                                remote_bind_address=('localhost', 3306),
                                local_bind_address=('localhost', 3300)):
            import UpdateDB as db
            case = db.updateresultOldCNNDiffBoth(
                doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                samples = samples, iterations = iterations, dropout = dropout, 
                train_acc_cnn = train_acc_cnn, val_acc_cnn = val_acc_cnn,
                test_acc_cnn = test_acc_cnn, test_bin_cnn = test_bin_cnn,
                train_acc_ml = train_acc_ml, val_acc_ml = val_acc_ml,
                test_acc_ml = test_acc_ml, test_bin_ml = test_bin_ml,
                test = test, port = 3300)
        
        import time
        time.sleep(10)

        from IPython.display import clear_output

        clear_output()

    else:
        print("Skipped: %12s" % (str(printstate)))

# import pandas as pd
# df = pd.DataFrame(output)
# df.to_csv("styloout.csv", index = False, encoding='utf-8')

import time
time.sleep(10)

Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 124
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (11578, 2)
Author:    44  Size:  4746
Author:    64  Size:  5106
Author:    82  Size:  1726
Min: 1726
Max: 5106
Authors [44, 64, 82].
Found 5178 texts.
Found 5178 labels.
Found 46523 unique tokens.
Shape of data tensor: (5178, 1000)
Shape of label tensor: (5178, 3)
Done compiling.
Done compiling.
Found 46523 unique tokens.
Shape of data tensor: (5178, 1000)
Shape of label tensor: (5178,)


Final Train Accuracy: 80.93

Final Validation Accuracy: 76.06
Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Found 46523 unique tokens.
Execution completed
Read completed
Number of rows: 1
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (43, 2)
Found 43 texts.
Found 46523 unique tokens.
Shape of data tensor: (43, 1000)
Done compiling.
Do

TypeError: not all arguments converted during string formatting

In [ ]:
%tb